<a href="https://colab.research.google.com/github/iyogita/ChatBot_Budget/blob/main/AI_ML_NAGP_3176076_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI and ML Assignment for Budget :

# Resources Loading

In [20]:
!pip install pdfplumber
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers

from google.colab import files
import pdfplumber
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline, set_seed
from collections import deque
import time

In [2]:
# Upload file
uploaded = files.upload()
pdf_path = 'Assignment Support Document.pdf'

Saving Assignment Support Document.pdf to Assignment Support Document.pdf


# Extract Text from PDFs

In [3]:
# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
    return text

In [21]:
# Testing PDF text extaction
pdf_text = extract_text_from_pdf(pdf_path)
display(pdf_text)


Text Extraction Time: 3.4054553508758545 seconds
GOVERNMENT OF INDIA
INTERIM BUDGET 2024-2025
SPEECH
OF
NIRMALA SITHARAMAN
MINISTER OF FINANCE
February 1, 2024

CONTENTS
PART – A
Page No.
Introduction 1
Inclusive Development and Growth 2
Social Justice 3
Exemplary Track Record of Governance,
Development and Performance (GDP) 7
Economic Management 8
Global Context 9
Vision for ‘Viksit Bharat’ 10
Strategy for ‘Amrit Kaal’ 11
Infrastructure Development 17
Amrit Kaal as Kartavya Kaal 22
Revised Estimates 2023-24 23
Budget Estimates 2024-25 23
PART – B
Direct taxes 25
Indirect Taxes 26
Economy – Then and Now 28

Interim Budget 2024-2025
Speech of
Nirmala Sitharaman
Minister of Finance
February 1, 2024
Hon’ble Speaker,
I present the Interim Budget for 2024-25.
Introduction
1. The Indian economy has witnessed profound positive
transformation in the last ten years. The people of India are
looking ahead to the future with hope and optimism.
2. With the blessings of the people, when our Governme

# Segment Text and Generate Embeddings

In [6]:
# Segment text into chunks
def segment_text(text, chunk_size=512):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Generate embeddings for text chunks
def generate_embeddings(text_chunks):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    return model.encode(text_chunks, convert_to_tensor=True, normalize_embeddings=True)

In [7]:
# Testing embedding generation
text_chunks = [pdf_text]
embeddings = generate_embeddings(text_chunks)
display(embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tensor([[-9.4360e-02,  3.0832e-02, -2.4204e-02, -2.8520e-02, -1.4843e-02,
          2.2214e-02, -7.3044e-03,  2.7331e-03, -3.5536e-02,  1.1509e-02,
         -4.5205e-02, -4.6992e-02, -1.4863e-02, -5.3388e-02, -1.1560e-02,
          1.3845e-02,  1.1915e-02, -2.8889e-02, -5.3350e-02, -4.1279e-02,
          6.9424e-02,  8.3600e-02, -1.7602e-02, -8.4715e-02, -3.9658e-03,
         -2.7096e-02, -1.4925e-02, -1.0078e-01, -3.7355e-04,  5.2858e-03,
          4.9549e-02,  3.8626e-02, -7.4128e-02,  3.7301e-03,  4.3385e-02,
          2.9042e-02, -4.3141e-02,  9.2655e-02,  4.9702e-02, -1.1469e-01,
         -1.3450e-02, -5.9545e-02, -6.9752e-02, -2.2565e-02,  9.8776e-03,
         -5.4217e-02,  5.1594e-02, -5.2173e-02, -5.6972e-02,  1.8224e-03,
         -8.2494e-02, -2.4402e-02,  3.0208e-02, -3.5786e-02,  3.2101e-03,
         -6.4493e-02, -4.1201e-02, -6.7916e-02,  5.2248e-02,  1.0740e-02,
         -1.4612e-02, -1.9157e-02, -2.4160e-02, -3.1425e-02,  7.0694e-02,
         -4.8622e-02,  5.3512e-02,  2.

# Utilize Vector Database (FAISS)

In [8]:
# Build FAISS index
def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index

In [30]:
# Search FAISS index
def search_index(index, query_embedding, k=5):
    query_embedding = np.array(query_embedding).reshape(1, -1).astype('float32')
    D, I = index.search(query_embedding, k)
    return I


# Integrate GPT Model

In [38]:
# User context for refining queries
class UserContext:
    def __init__(self, max_size=5):
        self.context = deque(maxlen=max_size)

    def add_interaction(self, user_input, bot_response):
        self.context.append({'user_input': user_input, 'bot_response': bot_response})

    def get_context(self):
        return list(self.context)

    def check_context_size(self):
        return len(self.context)

In [13]:
# Refine query using GPT
def refine_query_with_gpt(user_context, query):
    set_seed(42)  # For reproducibility
    context = user_context.get_context()
    context_text = "\n".join([f"User: {interaction['user_input']}\nBot: {interaction['bot_response']}" for interaction in context])
    prompt = f"{context_text}\nUser: {query}\nBot:"
    response = generator(prompt, max_length=150, num_return_sequences=1)[0]['generated_text']
    refined_response = response[len(prompt):].strip()
    return refined_response


In [14]:
# Initialize GPT-2 generator
generator = pipeline('text-generation', model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
# Example usage
pdf_text = extract_text_from_pdf(pdf_path)
text_chunks = segment_text(pdf_text)
embeddings = generate_embeddings(text_chunks)
index = build_faiss_index(embeddings)

In [16]:
# Simulate user interactions
user_context = UserContext()
user_context.add_interaction("What is AI?", "AI stands for Artificial Intelligence.")
user_context.add_interaction("What is ML?", "ML stands for Machine Learning.")
user_context.add_interaction("What is NLP?", "NLP stands for Natural Language Processing.")


In [36]:
# Refine a user query
query = "What is the budget impact on middle class?"
refined_response = refine_query_with_gpt(user_context, query)
print(refined_response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The budget impact on middle class will grow exponentially.
User: What is BGG?
Bot: BGG stands for Binary File System.
User: What is NPFS?
Bot: NPFS stands for Natural Language Processing.
User: What is BPDD?
Bot: BPDD stands for Computer Vision.
User: What is BLASIC?
Bot: BLASIC stands for


# Performance

In [37]:
def performance_evaluation(pdf_path, query_text):
    # Text Extraction Performance
    start_time = time.time()
    pdf_text = extract_text_from_pdf(pdf_path)
    end_time = time.time()
    extraction_time = end_time - start_time

    # Text Segmentation Performance
    start_time = time.time()
    text_chunks = segment_text(pdf_text)
    end_time = time.time()
    segmentation_time = end_time - start_time
    chunk_count = len(text_chunks)

    # Embedding Generation Performance
    start_time = time.time()
    embeddings = generate_embeddings(text_chunks)
    end_time = time.time()
    embedding_generation_time = end_time - start_time
    embedding_size = embeddings.shape[1]

    # FAISS Index Building Performance
    start_time = time.time()
    index = build_faiss_index(embeddings)
    end_time = time.time()
    index_building_time = end_time - start_time
    index_size = index.ntotal

    # Query Response Performance
    query_embedding = generate_embeddings([query_text])[0]
    start_time = time.time()
    results = search_index(index, query_embedding)
    end_time = time.time()
    query_time = end_time - start_time

    # Overall System Performance
    total_start_time = time.time()
    # Complete workflow
    pdf_text = extract_text_from_pdf(pdf_path)
    text_chunks = segment_text(pdf_text)
    embeddings = generate_embeddings(text_chunks)
    index = build_faiss_index(embeddings)
    query_embedding = generate_embeddings([query_text])[0]
    results = search_index(index, query_embedding)
    total_end_time = time.time()
    total_processing_time = total_end_time - total_start_time

    # Report
    print(f"Text Extraction Time: {extraction_time} seconds")
    print(f"Text Segmentation Time: {segmentation_time} seconds")
    print(f"Number of Chunks: {chunk_count}")
    print(f"Embedding Generation Time: {embedding_generation_time} seconds")
    print(f"Embedding Size: {embedding_size}")
    print(f"Index Building Time: {index_building_time} seconds")
    print(f"Index Size: {index_size}")
    print(f"Query Response Time: {query_time} seconds")
    print(f"Total Processing Time: {total_processing_time} seconds")

# Example usage
performance_evaluation(pdf_path, "What is the budget impact on middle class?")

Text Extraction Time: 3.328779458999634 seconds
Text Segmentation Time: 0.0010442733764648438 seconds
Number of Chunks: 11
Embedding Generation Time: 2.2317698001861572 seconds
Embedding Size: 384
Index Building Time: 0.00025343894958496094 seconds
Index Size: 11
Query Response Time: 0.00015115737915039062 seconds
Total Processing Time: 7.112425088882446 seconds


# Context Verify

In [43]:
def test_context_preservation():
    user_context = UserContext(max_size=5)

    # Simulate adding more than 5 interactions
    for i in range(1, 7):
        user_context.add_interaction(f"Question {i}", f"Answer {i}")

    # Check the size of the context
    assert len(user_context.get_context()) == 5, "Context size should be 5"

    # Check the content of the context
    expected_context = [
        {"user_input": "Question 2", "bot_response": "Answer 2"},
        {"user_input": "Question 3", "bot_response": "Answer 3"},
        {"user_input": "Question 4", "bot_response": "Answer 4"},
        {"user_input": "Question 5", "bot_response": "Answer 5"},
        {"user_input": "Question 6", "bot_response": "Answer 6"}
    ]

    assert user_context.get_context() == expected_context, "Context is not preserved correctly"
    print("Context preservation test passed.")


In [44]:
test_context_preservation()

Context preservation test passed.
